In [130]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get('https://icodrops.com/category/ended-ico/')
ICO_list = r.html.find('.col-md-12.col-12.a_ico')
ICO_page = [elmt.find('a') for elmt in ICO_list]
ICO_page = ICO_page[0:431]

ICO_list_page = []
for elmt in ICO_page :
    name = elmt[1].text
    page = elmt[1].attrs['href']
    if 'Pre-sale' not in name :
        if 'Presale' not in name :
            if 'All Sports' not in name :
                ICO_list_page += [[name,page]]


In [135]:
import csv
import datetime
with open('ICO_icodrops.csv', 'w', newline='') as csvfile:
    fieldnames = ['name', 'category', 'start', 'end', 'USD raised', 'USD aimed', 'ROI rate', 'accepted currency', 'video', 'website', 'whitepaper', 'facebook', 'twitter', 'instagram', 'linkedin', 'reddit', 'github', 'youtube channel', 'medium', 'bitcointalk', 'steemit', 'twitter link', 'github link', 'linkedin link', 'instagram link', 'youtube channel link']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for pos, page in enumerate(ICO_list_page) :
        r = session.get(page[1])
        table = r.html.find('article')
        table = table[0]
        data = { 'name' : page[0], 'start' : ' ', 'end' : ' ', 'category' : '', 'USD raised' : 0, 'USD aimed' : 0, 'ROI rate' : 'Normal', 'accepted currency' : ' ',
        'video': 0,
        'website': 0,
        'whitepaper': 0, 'facebook' : 0, 'twitter' : 0, 'instagram' : 0, 'linkedin' : 0, 'reddit' : 0, 'github' : 0, 'youtube channel' : 0, 'medium' : 0, 'bitcointalk' : 0, 'steemit' : 0, 'twitter link' : ' ', 'github link' : ' ', 'linkedin link' : ' ', 'youtube channel link' : ' ', 'instagram link' : ' '}
  
        
        
        ctgr = r.html.find('.ico-category-name')
        ctgr = ctgr[0].text.replace('(','')
        k = 0
        category = ''
        while ctgr[k] != ')' : 
            category += str(ctgr[k])
            k += 1
        data['category'] = category
        
        money = table.find('.fund-goal')
        money = money[0].text
        money = money.split("\n")
        if (len(money) == 3):
            raised = money[0]
            goal = money[2].split(' ')[0]
            raised = raised.replace(',','').replace('$','')
            raised = int(raised)
            goal = goal.replace(',','').replace('$','')
            goal = int(goal)
            data['USD raised'] += raised
            data['USD aimed'] += goal
        
        roi = table.find('.rating-box')
        if (len(roi) >= 3):
            roi = roi[2].text
            roi = roi.split('\n')
            roi = roi[1]
            data['ROI rate'] = roi

        video = table.find('iframe')
        if video:
            data['video'] += 1
    
        div = table.find('.ico-right-col')[0]
        div = div.find('.button')
        for btn in div:
            if 'website' in btn.text.lower():
                data['website'] += 1
            if 'whitepaper' in btn.text.lower():
                data['whitepaper'] += 1
                
                
        div = table.find('.ico-right-col')[0]
        div = div.find('.soc_links')
        div = div[0].find('a')
        for elmt in div :
            chaine = elmt.attrs['href']
            if 'facebook' in chaine :
                data['facebook'] += 1
            elif 'reddit' in chaine :
                data['reddit'] += 1
            elif 'twitter' in chaine :
                data['twitter'] += 1
                data['twitter link'] = str(elmt.attrs['href']).replace('https://twitter.com/','').replace('http://www.twitter.com/','').replace('?lang=en','').replace('@','').replace('/','')
            elif 'github' in chaine :
                data['github'] += 1
                data['github link'] = str(elmt.attrs['href']).replace('https://github.com/','')
            elif 'youtube' in chaine :
                data['youtube channel'] += 1
                data['youtube channel link'] = str(elmt.attrs['href'])
            elif 'instagram' in chaine :
                data['instagram'] += 1
            elif 'linkedin' in chaine :
                data['linkedin'] += 1
                data['linkedin link'] = str(elmt.attrs['href']).replace('https://www.linkedin.com/company/','').replace('/','')
            elif 'medium' in chaine :
                data['medium'] += 1
            elif 'bitcointalk' in chaine :
                data['bitcointalk'] += 1
            elif 'steemit' in chaine :
                data['steemit'] += 1
        
        
        year = datetime.timedelta(365)
        
        if pos <= 208 :
            annee = str(2018)
        else :
            annee = str(2017)
            
        r = table.find('.col-12.title-h4')
        for elmt in r : 
            r = elmt.find('h4')
            if 'Token' in r[0].text :
                period = str(r[0].text).replace('Token Sale:','').replace(' ','')
                if '(noexactdates)' in period : 
                    period = period.replace('(noexactdates)','')
                    if '-' in period:
                        start= period[0:3]
                        k = 3
                        while period[k] != '-' :
                            k += 1
                        k += 1
                        end = period[k:k+3]
                        start += str(annee)
                        end += str(annee)
                        start = datetime.datetime.strptime(start,'%b%Y')
                        end = datetime.datetime.strptime(end,'%b%Y')
                        if start > end :
                            start -= year
                        data['start'] = str(start).replace(' 00:00:00','')
                        data['end'] = str(end).replace(' 00:00:00','')
                
                elif 'until' in period :
                    period = period.replace('until', '').replace(' ','')
                    end = period + str(annee)
                    data['end'] = str(datetime.datetime.strptime(end,'%d%b%Y')).replace(' 00:00:00','')
                
                else : 
                    start = ''
                    end = ''
                    k = 0
                    while period[k] != '–':
                        start += period[k]
                        k += 1 
                    k += 1 
                    end = period[k:]
                    start += str(annee)
                    end += str(annee)
                    start = datetime.datetime.strptime(start,'%d%b%Y')
                    end = datetime.datetime.strptime(end,'%d%b%Y')
                    if start > end :
                        start -= year
                    data['start'] = str(start).replace(' 00:00:00','')
                    data['end'] = str(end).replace(' 00:00:00','')
        
        
        
        r = table.find('.col-12.col-md-6')[1]
        r = r.find('li')
        if len(r) ==6 and r[5] != [] :
            accepted_currency = str(r[5].text).replace('Accepts: ','')
            data['accepted currency'] = str(accepted_currency)

        writer.writerow(data)
        